In [1]:
from os.path import exists

In [2]:
def run(data_size, suffix="", ep=3):
    if exists('./checkpoints'):
        !rm -r ./checkpoints
    
    train_set_suffix = f"_{data_size}{suffix}"
    train_set_path = './DATA/train' + train_set_suffix + '.txt' 

    !python -u -m paddle.distributed.launch --gpus "0" train_pointwise.py \
            --train_set_path {train_set_path} \
            --dev_set_path './dev.txt' \
            --epochs {ep} \
            --save_dir ./checkpoints\
            --batch_size 64 \
            --learning_rate 2E-5 \
    
    model = !ls ./checkpoints/
    model.sort()
    model_path = f"./checkpoints/{model[-1]}/model_state.pdparams"

    !python -u -m paddle.distributed.launch --gpus "0" \
            predict_pointwise.py \
            --out_dir './ablation_tests_pred/' \
            --train_set_suffix {train_set_suffix} \
            --device gpu \
            --params_path {model_path}\
            --batch_size 64 \
            --max_seq_length 128 \
            --input_file 'test.txt'
    

In [1]:
from IPython.display import clear_output

data_size = ['5k', '10k', '25k', '50k', '75k', '100k', '125k', '150k', '175k', '200k', 'full']
suffix = ['', '_sr_reda', '_rs_reda', '_ri_reda', '_rd_reda', '_rm_reda', 
'_sr_reda_ngram', '_rs_reda_ngram', '_ri_reda_ngram', '_rd_reda_ngram', '_rm_reda_ngram']

for sfx in suffix:
    for size in data_size:
        if size in ['5k', '10k', '25k']:
            ep = 3
        elif size in ['50k', '75k', '100k']:
            ep = 2
        else:
            ep = 1

        run(size, sfx, ep)
        clear_output(wait=True)